------ standard imports ------ #

In [ ]:
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
import cv2 as cv

In [ ]:
import ansitable
ansitable.options(unicode=True)

In [ ]:
from spatialmath import *
from spatialmath.base import *
BasePoseMatrix._color=False
from roboticstoolbox import *

In [ ]:
from spatialmath.base import *
import math
from math import pi

In [ ]:
from machinevisiontoolbox import *
from machinevisiontoolbox.base import *

In [ ]:
np.set_printoptions(
    linewidth=120, formatter={
        'float': lambda x: f"{0:8.4g}" if abs(x) < 1e-10 else f"{x:8.4g}"})

In [ ]:
np.random.seed(0)
cv.setRNGSeed(0)

------------------------------ #

Perspective Camera

Perspective Projection

Modeling a Perspective Camera

In [ ]:
camera = CentralCamera(f=0.015);
P = [0.3, 0.4, 3.0];
camera.project_point(P)
camera.project_point(P, pose=SE3.Tx(-0.5))

Discrete Image Plane

In [ ]:
camera = CentralCamera(f=0.015, rho=10e-6,
 imagesize=[1280, 1024], pp=[640, 512], name="mycamera")
camera.project_point(P)

Camera Matrix

In [ ]:
camera.K
camera.C()
np.rad2deg(camera.fov())
P = np.column_stack([[0, 0, 10], [10, 10, 10]])
p, visible = camera.project_point(P, visibility=True)
visible

Projecting Points

In [ ]:
P = mkgrid(n=3, side=0.2, pose=SE3.Tz(1.0));
P.shape
P[:, :4]
camera.project_point(P)
camera.plot_point(P);
T_camera = SE3.Trans(-1, 0, 0.5) * SE3.Ry(0.9);
camera.plot_point(P, pose=T_camera);
camera.project_point([1, 0, 0, 0], pose=T_camera)
camera.clf()  # clear the virtual image plane
p = camera.plot_point(P, pose=T_camera)
cube = mkcube(0.2, pose=SE3.Tz(1));
cube.shape
camera.clf()  # clear the virtual image plane
camera.plot_point(cube);
X, Y, Z = mkcube(0.2, pose=SE3.Tz(1), edge=True)
X.shape
camera.plot_wireframe(X, Y, Z)
camera.clf()  # clear the virtual image plane
camera.plot_wireframe(X, Y, Z, pose=T_camera);

Lens Distortion

Camera Calibration

Calibrating with a 3D Target

In [ ]:
P = mkcube(0.2);
T_unknown = SE3.Trans(0.1, 0.2, 1.5) * SE3.RPY(0.1, 0.2, 0.3);
camera_unknown = CentralCamera(f=0.015, rho=10e-6, imagesize=[1280, 1024], noise=0.05, seed=0)
p = camera_unknown.project_point(P, objpose=T_unknown);
C, resid = CentralCamera.points2C(P, p)
C
resid

Calibrating with a Checkerboard

In [ ]:
images = ImageCollection("calibration/*.jpg");
len(images)
K, distortion, frames = CentralCamera.images2C(images, gridshape=(7,6), squaresize=25e-3)
K
images[0].centre
for frame in frames:
  CentralCamera.plot(pose=frame.pose, scale=0.05)
distortion

Correcting for Lens Distortion

In [ ]:
u0 = K[0, 2]; v0 = K[1, 2]; fpix_u = K[0, 0]; fpix_v = K[1,1];
k1, k2, p1, p2, k3 = distortion;
U, V = images[12].meshgrid()
u = (U - u0) / fpix_u;
v = (V - v0) / fpix_v;
r = np.sqrt(u**2 + v**2);
delta_u = u * (k1*r**2 + k2*r**4 + k3*r**6) + p1*u*v + p2*(r**2 + 2*u**2);
delta_v = v * (k1*r**2 + k2*r**4 + k3*r**6) + p1*(r**2 + 2*v**2) + p2*u*v;
ud = u + delta_u; vd = v + delta_v;
Ud = ud * fpix_u + u0;
Vd = vd * fpix_v + v0;
undistorted = images[12].warp(Ud, Vd)
plt.clf()   # clear 3D plot
plt.quiver(Ud[::50, ::50], Vd[::50, ::50], -delta_u[::50, ::50], -delta_v[::50, ::50]);
magnitude = np.sqrt(delta_u**2 + delta_v**2);
plt.contour(U, V, magnitude);

Decomposing the Camera Calibration Matrix

In [ ]:
o = sp.linalg.null_space(C);
o.T
h2e(o).T
T_unknown.inv().t
est = CentralCamera.decomposeC(C)
est.f / est.rho[0]
camera.f / camera.rho[0]
(T_unknown * est.pose).printline(orient="camera")
plotvol3([-0.9, 0.9, -0.9, 0.9, -1.5, 0.3]);
plot_sphere(0.03, P, color="r");
SE3().plot(frame="T", color="b", length=0.3);
est.plot(scale=0.3, color="black", frame=True);

Pose Estimation with a Calibrated Camera

In [ ]:
camera_calib = CentralCamera.Default(noise=0.1, seed=0);
P = mkcube(0.2);
T_unknown = SE3.Trans(0.1, 0.2, 1.5) * SE3.RPY(0.1, 0.2, 0.3);
T_unknown.printline()
p = camera_calib.project_point(P, objpose=T_unknown);
T_est = camera_calib.estpose(P, p).printline()

Wide Field-of-View Cameras

Fisheye Lens Camera

In [ ]:
camera = FishEyeCamera(
          projection="equiangular",
          rho=10e-6,
          imagesize=[1280, 1024]
          );
X, Y, Z = mkcube(side=1, centre=[1, 1, 0.8], edge=True);
camera.plot_wireframe(X, Y, Z, color="k");

Catadioptric Camera

In [ ]:
camera = CatadioptricCamera(
          projection="equiangular",
          rho=10e-6,
          imagesize=[1280, 1024],
          maxangle=pi/4
      );
X, Y, Z = mkcube(1, centre=[1, 1, 0.8], edge=True)
camera.plot_wireframe(X, Y, Z, color="k");

Spherical Camera

In [ ]:
camera = SphericalCamera()
X, Y, Z = mkcube(1, centre=[2, 3, 1], edge=True)
camera.plot_wireframe(X, Y, Z, color="k");

Unified Imaging Model

Mapping Wide-Angle Images to the Sphere

In [ ]:
u0 = 528.1214; v0 = 384.0784; l = 2.7899; m = 996.4617;
fisheye = Image.Read("fisheye_target.png", dtype="float", mono=True)
n = 500;
phi_range = np.linspace(-pi, pi, n);  # longitude
theta_range = np.linspace(0, pi, n);     # colatitude
Phi, Theta = np.meshgrid(phi_range, theta_range);
r = (l + m) * np.sin(Theta) / (l - np.cos(Theta));
U = r * np.cos(Phi) + u0;
V = r * np.sin(Phi) + v0;
spherical = fisheye.warp(U, V, domain=(phi_range, theta_range))
spherical.disp(axes=("$\phi$", "$\theta$"));
ax = plotvol3();
plot_sphere(radius=1, ax=ax, filled=True, resolution=n,
  facecolors=spherical.colorize().A, cstride=1, rstride=1);

Mapping from the Sphere to a Perspective Image

In [ ]:
W = 1000;
m = W / 2 / np.tan(np.deg2rad(45 / 2))
l = 0;
u0, v0 = W / 2, W / 2;
U, V = Image.meshgrid(width=W, height=W);
U0 = U - u0; V0 = V - v0;
r = np.sqrt(U0**2 + V0**2);
phi = np.arctan2(V0, U0);
Phi = phi;
Theta = pi - np.arctan(r / m);
spherical.warp(Phi, Theta).disp();
spherical2 = spherical.rotate_spherical(SO3.Ry(0.9) * SO3.Rz(-1.5))
spherical2.warp(Phi, Theta).disp();

Novel Cameras

Multi-Camera Arrays

Light-Field Cameras

Applications

Fiducial Markers

In [ ]:
scene = Image.Read("lab-scene.png", rgb=False)
scene.disp();
camera = CentralCamera(f=3045, imagesize=scene.shape,
                       pp=(2016, 1512), rho=1.4e-6);
markers = scene.fiducial(dict="4x4_50", K=camera.K, side=67e-3);
markers[2]
markers[2].corners
for marker in markers:
  marker.draw(scene, length=0.10, thick=20)

Planar Homography

In [ ]:
T_camera = SE3.Tz(8) * SE3.Rx(-2.8);
camera = CentralCamera.Default(f=0.012, pose=T_camera);
P = np.column_stack([[-1, 1], [-1, 2], [2, 2], [2, 1]])
camera.project_point(np.vstack([P, np.zeros((4,))]))
H = np.delete(camera.C(), 2, axis=1)
homtrans(H, P)
p = np.column_stack([[0, 0], [0, 1000], [1000, 1000], [1000, 0]])
Pi = homtrans(np.linalg.inv(H), p)
camera.plot(scale=2, color="black");
plot_sphere(radius=0.1, centre=np.vstack((P, np.zeros((4,)))), color="red");
plot_sphere(radius=0.1, centre=np.vstack((Pi, np.zeros((4,)))), color="blue");

Advanced Topics

Projecting 3D Lines and Quadrics

In [ ]:
L = Line3.Join((0, 0, 1), (1, 1, 1))
L.w
camera = CentralCamera.Default();
l = camera.project_line(L)
camera.plot_line2(l)
camera.plot_line3(L)
T_camera = pose=SE3.Trans(0.2, 0.1, -5) * SE3.Rx(0.2);
camera = CentralCamera.Default(f=0.015, pose=T_camera);
Q = np.diag([1, 1, 1, -1]);
adj = lambda A: np.linalg.det(A) * np.linalg.inv(A);
C = camera.C();
c = adj(C @ adj(Q) @ C.T)
np.linalg.det(c[:2, :2])
from sympy import symbols, Matrix, Eq, plot_implicit
x, y = symbols("x y")
X = Matrix([[x, y, 1]]);
ellipse = X * Matrix(c) * X.T;
plot_implicit(Eq(ellipse[0], 1), (x, 0, 1_000), (y, 0, 1_000), );

Nonperspective Cameras

Wrapping Up

Further Reading and Resources

Exercises